# Convert HDH5 to npz format

> Joseph P. Vantassel, The University of Texas at Austin

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

## Load hdf5 and store in list of ndarrays

In [ ]:
numbers = range(0, 100000, 200)
positions = []

for number in numbers:
    fname = f"mpm_1to1/particles{str(number).zfill(6)}.h5"
    position = np.empty((1024, 2), dtype=float)
    with h5py.File(fname, "r") as f:
        position[:, 0] = f["table"]["coord_x"][:]
        position[:, 1] = f["table"]["coord_y"][:]
    positions.append(position)

positions = np.array(positions)
print(positions.shape)

## Calculate Mean Velocity and Mean Acceleration

In [ ]:
numbers = range(0, 100000, 200)
velocities = []
accelerations = []

for number in numbers:
    fname = f"mpm_1to1/dataset/raw/particles{str(number).zfill(6)}.h5"
    velocity = np.empty((1024, 2), dtype=float)
    acceleration = np.empty((1024, 2), dtype=float)
    with h5py.File(fname, "r") as f:
        velocity[:, 0] = f["table"]["velocity_x"][:]
        velocity[:, 1] = f["table"]["velocity_y"][:]
        
        acceleration[:, 0] = f["table"]["velocity_x"][:]
        acceleration[:, 1] = f["table"]["velocity_y"][:]
    velocities.append(velocity)
    accelerations.append(acceleration)

velocities = np.array(velocities)
accelerations = np.array(accelerations)

print(f"Velcocity: mean={np.mean(velocities, axis=(0,1))}, std={np.std(velocities, axis=(0,1))}")
print(f"Acceleration: mean={np.mean(accelerations, axis=(0,1))}, std={np.std(accelerations, axis=(0,1))}")

## Plot trajectory

In [ ]:
%matplotlib qt5

fig, ax = plt.subplots(figsize=(5,5), dpi=150)
plt.ion()
fig.show()

for position in positions:
    ax.clear()
    ax.scatter(position[:, 0], position[:, 1], s=0.5)
    ax.set_xlim(0., 1.)
    ax.set_ylim(0., 1.)
    plt.pause(.000001)
    fig.canvas.draw()

plt.close()  

## Save to `npz`

In [ ]:
trajectories = {}
trajectories["simulation_trajectory_0"] = (positions, np.full(positions.shape[1], 6, dtype=int))
np.savez_compressed("train_mpm1to1.npz", **trajectories)

## Check Conversion to `npz`

In [ ]:
a = np.load("train_mpm1to1.npz", allow_pickle=True)
a["simulation_trajectory_0"]